# Making the Reference Genome more easily usable

> "Using BioPython to convert the reference human genome into a sqlite database."

In [ ]:
#| default_exp features.genome

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
from Bio import SeqIO
import sqlite3

/home/jdb/projects/llm-mito-scanner/venv/lib/python3.10/site-packages/Bio/__init__.py:138: BiopythonWarning: You may be importing Biopython from inside the source tree. This is bad practice and might lead to downstream issues. In particular, you might encounter ImportErrors due to missing compiled C extensions. We recommend that you try running your code from outside the source tree. If you are outside the source tree then you have a setup.py file in an unexpected directory: /home/jdb/projects/llm-mito-scanner/venv/lib/python3.10/site-packages
  warnings.warn(


In [ ]:
#| hide
# !gzip ../data/raw/GRCh38_latest_genomic.fna.gz -d

In [ ]:
#| hide
# !gzip ../data/raw/GRCh38_latest_genomic.gff.gz -d

In [ ]:
#| hide
data_path = Path("../data")
genome_file_path = data_path / "raw/GRCh38_latest_genomic.fna"
annotations_file_path = data_path / "raw/GRCh38_latest_genomic.gff"

In [ ]:
def get_or_make_sqlite_connection(data_path: Path) -> sqlite3.Connection:
    """
    Get or make a sqlite database connection.
    """
    return sqlite3.connect(data_path / "genome.db")

def sqlite_table_exists(table_name: str, connection: sqlite3.Connection) -> bool:
    """
    Return True is table exists in sqlite database, else False.
    """
    cursor = connection.cursor()
    result = cursor.execute(f"SELECT name FROM sqlite_master WHERE name='{table_name}'")
    table_exists = result.fetchone()
    cursor.close()
    return table_exists is not None

def setup_sqlite(connection: sqlite3.Connection, overwrite: bool = False):
    cursor = connection.cursor()
    # Tables
    if not sqlite_table_exists("sequence", connection):
        cursor.execute("CREATE TABLE sequence(id, name, description, sequence)")
    if not sqlite_table_exists("sequence_idx", connection):
        cursor.execute("CREATE UNIQUE INDEX sequence_idx ON sequence(id)")
    cursor.close()

In [ ]:
#| hide
conn = get_or_make_sqlite_connection(data_path)
setup_sqlite(conn)

In [ ]:
#| export
def format_sequence_for_insert(sequence_record: SeqIO.SeqRecord) -> str:
    """
    Format a SeqIO.SeqRecord for insertion to the sqlite database.
    """
    return f"('{sequence_record.id}', '{sequence_record.name}', '{sequence_record.description}', '{sequence_record.seq}')"


def insert_sequences(records: list[SeqIO.SeqRecord], connection: sqlite3.Connection):
    """
    Insert a batch of SeqIO.SeqRecords into the sqlite database.
    """
    values_str = ",".join(map(format_sequence_for_insert, records))
    cursor = connection.cursor()
    try:
        cursor.execute(
            f"""
            INSERT INTO sequence VALUES {values_str} 
            ON CONFLICT(id) DO NOTHING;
            """
        )
    except Exception as e:
        raise e
    finally:
        cursor.close()
    

def insert_genome_into_sqlite(
    genome_file_path: Path, 
    sqlite_connection: sqlite3.Connection, 
    insert_batch_size: int = 10
):
    """
    Insert the fasta-annotated sequence records from our reference genome 
    into our sqlite3 database.
    """
    records = []
    for i, seq_record in enumerate(SeqIO.parse(genome_file_path, "fasta")):
        records.append(seq_record)
        if i % insert_batch_size == 0:
            insert_sequences(records, sqlite_connection)
            records = []
    insert_sequences(records, sqlite_connection)

In [ ]:
#| hide
insert_genome_into_sqlite(genome_file_path, conn)

In [ ]:
#| hide
cursor = conn.cursor()

In [ ]:
#| hide
num_rows = cursor.execute("SELECT COUNT(DISTINCT ID) FROM sequence;").fetchone()[0]
assert num_rows > 0

In [ ]:
#| hide
num_rows

705

In [ ]:
#| hide
test_sequence = cursor.execute("SELECT * FROM sequence LIMIT 1;").fetchone()

In [ ]:
#| hide
# ID is string of non-zero length
assert isinstance(test_sequence[0], str) and len(test_sequence[0]) > 0
test_sequence[0]

'NC_000001.11'

In [ ]:
#| hide
# Name is string
assert isinstance(test_sequence[1], str)

In [ ]:
#| hide
test_sequence[1]

'NC_000001.11'

In [ ]:
#| hide
# Description is string
assert isinstance(test_sequence[2], str)

In [ ]:
#| hide
test_sequence[2]

'NC_000001.11 Homo sapiens chromosome 1, GRCh38.p14 Primary Assembly'

In [ ]:
#| hide
# Sequence is string and of non-zero length
assert isinstance(test_sequence[3], str) and len(test_sequence[3]) > 0

In [ ]:
#| hide
len(test_sequence[3])

248956422

In [ ]:
#| hide
from collections import Counter

test_sequence_counter = Counter(test_sequence[3])
# test_sequence_counter.keys()

In [ ]:
#| hide
test_sequence_counter["N"]

18475408

In [ ]:
#| hide
test_sequence_counter

Counter({'T': 41321614,
         'A': 41260191,
         'C': 31266202,
         'G': 31256045,
         't': 25922550,
         'a': 25810086,
         'N': 18475408,
         'g': 16855483,
         'c': 16788841,
         'M': 1,
         'R': 1})

In [ ]:
#| hide
test_sequence[3][:10], test_sequence[3][-10:]

('NNNNNNNNNN', 'NNNNNNNNNN')

In [ ]:
#| hide
cursor.close()

In [ ]:
#| hide
conn.close()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()